Training deeper networks using residual connections. In this notebook, a 50-layer residual network popularly known as ResNet50, is built from scratch and trained on a custom dataset.

In [4]:
import numpy as np
import tensorflow as tf
tf.__version__

'2.5.0'

In [5]:
def block_identity(X, num_filters, k):

    num1, num2, num3 = num_filters
    X_skip = X

    X = tf.keras.layers.Conv2D(filters=num1, kernel_size=(1,1), padding='valid')(X)
    X = tf.keras.layers.BatchNormalization()(X)
    X = tf.keras.layers.Activation('relu')(X)

    X = tf.keras.layers.Conv2D(filters=num2, kernel_size=(k,k), padding='same')(X)
    X = tf.keras.layers.BatchNormalization()(X)
    X = tf.keras.layers.Activation('relu')(X)

    X = tf.keras.layers.Conv2D(filters=num3, kernel_size=(1,1), padding='valid')(X)
    X = tf.keras.layers.BatchNormalization()(X)

    X = tf.keras.layers.Add()([X_skip, X])
    X = tf.keras.layers.Activation('relu')(X)

    return X

In [8]:
def block_conv(X, num_filters, k, s):

    num1, num2, num3 = num_filters
    X_skip = X

    X = tf.keras.layers.Conv2D(filters=num1, kernel_size=(1,1), strides=(s,s), padding='valid')(X)
    X = tf.keras.layers.BatchNormalization()(X)
    X = tf.keras.layers.Activation('relu')(X)

    X = tf.keras.layers.Conv2D(filters=num2, kernel_size=(k,k), strides=(1,1), padding='same')(X)
    X = tf.keras.layers.BatchNormalization()(X)
    X = tf.keras.layers.Activation('relu')(X)

    X = tf.keras.layers.Conv2D(filters=num3, kernel_size=(1,1), strides=(1,1), padding='valid')(X)
    X = tf.keras.layers.BatchNormalization()(X)

    X_skip = tf.keras.layers.Conv2D(filters=num3, kernel_size=(1,1), strides=(s,s), padding='valid')(X_skip)
    X_skip = tf.keras.layers.BatchNormalization()(X_skip)

    X = tf.keras.layers.Add()([X_skip, X])
    X = tf.keras.layers.Activation('relu')(X)

    return X 

In [9]:
def resnet50(input_shape = (64, 64, 3), classes = 6):
    
    X_input = tf.keras.layers.Input(input_shape)

    X = tf.keras.layers.ZeroPadding2D((3, 3))(X_input)
    
    X = tf.keras.layers.Conv2D(64, (7, 7), strides = (2, 2))(X)
    X = tf.keras.layers.BatchNormalization()(X)
    X = tf.keras.layers.Activation('relu')(X)
    X = tf.keras.layers.MaxPooling2D((3, 3), strides=(2, 2))(X)

    X = block_conv(X, num_filters = [64, 64, 256], k = 3, s=1)
    X = block_identity(X, num_filters=[64, 64, 256], k=3)
    X = block_identity(X, num_filters=[64, 64, 256], k=3)

    X = block_conv(X, num_filters = [128, 128, 512], k = 3, s = 2)
    X = block_identity(X, num_filters=[128, 128, 512], k=3)
    X = block_identity(X, num_filters=[128, 128, 512], k=3)
    X = block_identity(X, num_filters=[128, 128, 512], k=3)

    X = block_conv(X, num_filters = [256, 256, 1024], k = 3, s = 2)
    X = block_identity(X, num_filters=[256, 256, 1024], k=3)
    X = block_identity(X, num_filters=[256, 256, 1024], k=3)
    X = block_identity(X, num_filters=[256, 256, 1024], k=3)
    X = block_identity(X, num_filters=[256, 256, 1024], k=3)
    X = block_identity(X, num_filters=[256, 256, 1024], k=3)

    X = block_conv(X, num_filters=[512, 512, 2048], k=3, s=2)
    X = block_identity(X, num_filters=[512, 512, 2048], k=3)
    X = block_identity(X, num_filters=[512, 512, 2048], k=3)

    X = tf.keras.layers.AveragePooling2D(pool_size=(2,2))(X)

    X = tf.keras.layers.Flatten()(X)
    X = tf.keras.layers.Dense(classes, activation='softmax')(X)

    model = tf.keras.Model(inputs = X_input, outputs = X)

    return model

In [10]:
model = resnet50(input_shape=(64,64,3), classes=6)
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])


In [24]:
nclasses = 6
import h5py

def load_dataset():
    train_set = h5py.File('/content/train_signs.h5', "r")
    x_train_orig = np.array(train_set["train_set_x"][:])
    y_train_orig = np.array(train_set["train_set_y"][:])

    test_set = h5py.File('/content/test_signs.h5', "r")
    x_test_orig = np.array(test_set["test_set_x"][:])
    y_test_orig = np.array(test_set["test_set_y"][:])

    labels = np.array(test_set["list_classes"][:])
    
    return x_train_orig, x_test_orig, y_train_orig, y_test_orig, labels

x_train_orig, x_test_orig, y_train_orig, y_test_orig, labels = load_dataset()
x_train = x_train_orig/255.
x_test = x_test_orig/255.
print(x_train.shape, x_test.shape)
# (x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()
# x_train = x_train/255.
# x_test = x_test/255.
y_train_oh = tf.one_hot(y_train_orig, depth=nclasses, on_value=1., off_value=0.)
y_test_oh = tf.one_hot(y_test_orig, depth=nclasses, on_value=1., off_value=0.)
print(y_train_oh.shape)

(1080, 64, 64, 3) (120, 64, 64, 3)
(1080, 6)


In [27]:
print(y_test_oh[0])
print(y_test_orig[0])

tf.Tensor([1. 0. 0. 0. 0. 0.], shape=(6,), dtype=float32)
0


In [28]:
model.fit(x_train, y_train_oh, epochs=15, batch_size=32)

Epoch 1/15
34/34 [==============================] - 2s 59ms/step - loss: 2.0660e-04 - accuracy: 1.0000
Epoch 2/15
34/34 [==============================] - 2s 57ms/step - loss: 1.1544e-04 - accuracy: 1.0000
Epoch 3/15
34/34 [==============================] - 2s 56ms/step - loss: 2.8389e-04 - accuracy: 1.0000
Epoch 4/15
34/34 [==============================] - 2s 56ms/step - loss: 2.4149e-04 - accuracy: 1.0000
Epoch 5/15
34/34 [==============================] - 2s 56ms/step - loss: 5.9791e-05 - accuracy: 1.0000
Epoch 6/15
34/34 [==============================] - 2s 56ms/step - loss: 5.3049e-05 - accuracy: 1.0000
Epoch 7/15
34/34 [==============================] - 2s 56ms/step - loss: 7.1671e-05 - accuracy: 1.0000
Epoch 8/15
34/34 [==============================] - 2s 57ms/step - loss: 6.7012e-05 - accuracy: 1.0000
Epoch 9/15
34/34 [==============================] - 2s 57ms/step - loss: 6.0164e-05 - accuracy: 1.0000
Epoch 10/15
34/34 [==============================] - 2s 56ms/step - loss:

In [29]:
model.evaluate(x_test,y_test_oh)

4/4 [==============================] - 0s 28ms/step - loss: 0.1759 - accuracy: 0.9583


[0.17588749527931213, 0.9583333134651184]